In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import re
import sys
import time
import requests
import math

keys = Keys()

In [2]:
# webdriver 자동설정 (M1)
def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

In [4]:
# 제품별 정보/평점/리뷰 크롤링을 위한, url 패턴 product number 리스트 만들기

list_prod_num = []    # -> 제품별 정보 크롤링 페이지에서 url 패턴에 사용
for page in range(1, 2): #80개기준 5페이지
    url =f'https://smartstore.naver.com/woorisulgallery/category/2a59932df99946be91f799d367bf2f64?st=RECENT&free=false&dt=LIST&subscr=false&page={page}&size=80'
    header = {
         # referer 유저가 이전에 있던 위치
        'referer': 'https://smartstore.naver.com/woorisulgallery/products/7465928721',
        # 내 노트북 정보. Network > Headers의 user_agent 복사
        'user_agent' : 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Mobile Safari/537.36',
    }
    resp = requests.get(url, headers = header)
    html = BeautifulSoup(resp.text,'html.parser')
    print(f'{page}page crwaling')
    
    for item in (html.select('a._1vVKEk_wsi')):
        list_prod_num.append(item['href'][26:])    #url에서 제품별 고유번호 뽑아내고 이거 리스트에 넣어서 패턴에 돌리도록.

    #차단 막는 코드. 랜덤으로 시간차 준다.
    seed = np.random.randint(100)
    np.random.seed(seed)  
    a = np.random.randint(5)
    time.sleep(a)

1page crwaling


In [6]:
list_prod_num

['7465928721',
 '7420531754',
 '7407760829',
 '7375151215',
 '7230151129',
 '7223263131',
 '7222458593',
 '7181582494',
 '7091201292',
 '7091141602',
 '7086916801',
 '7082141168',
 '7082114868',
 '7081787063',
 '7081727453',
 '7023638208',
 '6995602257',
 '6989342630',
 '6989293515',
 '6985389412',
 '6985318181',
 '6985010996',
 '6984873701',
 '6984692409',
 '6970201049',
 '6970175872',
 '6969789066',
 '6965324921',
 '6965207541',
 '6965150027',
 '6965096487',
 '6944811227',
 '6942459524',
 '6942455044',
 '6942450071',
 '6938133936',
 '6938099624',
 '6938081471',
 '6934386825',
 '6934377719',
 '6934324943',
 '6919491286',
 '6901842665',
 '6886996431',
 '6845416112',
 '6817989449',
 '6817629591',
 '6817597428',
 '6817581846',
 '6817567179',
 '6808549888',
 '6779899202',
 '6779841775',
 '6744727889',
 '6737478231',
 '6721417485',
 '6721406477',
 '6712052250',
 '6692488489',
 '6692342914',
 '6692327583',
 '6692313605',
 '6692302086',
 '6692270739',
 '6692241821',
 '6692226646',
 '66921999

## 모든 제품 페이지 크롤링하는 코드

In [77]:
for num in (list_prod_num):
    
    url =f'https://smartstore.naver.com/woorisulgallery/products/{num}'

    header = {
         # referer --> 유저가 이전에 있던 위치를 알려주는 거, 안써주면 네이버가 정보 안준다
        'referer': 'https://finance.daum.net/quotes/A005930',
        # 나 기계아니고 사람임~ 내 노트북 정보
        'user_agent' : 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Mobile Safari/537.36',
    }
    #헤더 정보를 넣어주기
    resp = requests.get(url, headers = header)

    driver = set_chrome_driver()
    driver.get(url)
    time.sleep(2)

    # 제품 정보 뽑아내기
    resp.text
    html_text = BeautifulSoup(resp.text, 'html.parser')
    product_info = [item.text.strip().replace('\t', '')  for item in html_text.select('div.CxNYUPvHfB')]
    print(product_info[0])

    # 리뷰가 있을 경우만 크롤링
    if int(html_text.select('strong._2pgHN-ntx6')[0].text) < 1 :    # 리뷰 1개 이하이면 pass
        pass

    else :
        count = 1
        stop = math.ceil(int(html_text.select('strong._2pgHN-ntx6')[0].text) / 20)  # 전체 리뷰 수 / 한 페이지 리뷰수 20개 

        review_list = []
        # 맨처음 리뷰 1페이지 누르고
        wait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#REVIEW > div > div._180GG7_7yx > div.cv6id6JEkg > div > div > a:nth-child(2)'))).click()
        while count < stop+1:
            # 페이지에 리뷰 수 구하기 위한 코드 : len() 
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            review = soup.find_all('div',class_='_1YShY6EQ56')
            for i in range(len(review)):
                review = soup.find_all('div',class_='_1YShY6EQ56')
                review = review[i].text
                review = re.sub('[^#0-9a-zA-Zㄱ-ㅣ가-힣 ]',"",review)
                review_list.append(review)        
                # print(i, review)
            print(f'{count} 페이지 크롤링 완료 !!')
            # 다음페이지 누르기
            if count == stop:
                break
            else:
                wait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#REVIEW > div > div._180GG7_7yx > div.cv6id6JEkg > div > div > a.fAUKm1ewwo._2Ar8-aEUTq'))).click()
                time.sleep(2)
                count = count + 1


내변산 양조장 백제소주 25도375ml 증류식소주
0 평점5st221103신고오랜만에 증류주를 먹어보네요 쌀의 은은한 향이아주좋습니다
1 평점5fixi221031신고도수에 비해 알콜이 많이 느껴지지않고 아주 부드럽습니다풍미는 생각보다는 심심한편입니다
2 평점5yimi221101신고잘왔네요 잘먹을께요 감사합니다
3 평점5ec221021신고약간의 기대로 마셨는데 기대 이상으로 괜찮은 술이었습니다 앞으로 자주 먹게 될 것 같네요
4 평점4dion221029신고술도 사은품도 빠르게 잘받았습니다 감사합니다
5 평점5hpar221025신고배송도 빠르고 안전하게 잘받았습니다 아직 먹기는 전이지만 잘 먹겠습니다
6 평점5kjh2221030신고배송은 하루만에 도착했구요맛은 몇번 경험해서 이제는 익숙하네요 목넘김도 부드럽고 다음날 개운해서좋네요 자주 애용하겠습니다
7 평점5qs221024신고술 자체의 풍미가 좋았습니다 증류싁 소주의 장점이 부드러운 목넘김은 당연하구요
8 평점5eseu221021신고요즘 증류주에 빠저있는데 단연 최고내요 또 주문해야겠습니다
9 평점5abcd221031신고도수가 좀 있는데도 향도 좋고 부드러웠습니다
10 평점5jyyo221028신고타격감이 있고 부트러운게 맛난 증류주네요
11 평점5hl220922신고요즘 우리술에 관심생겨서 여러가지 맛보는중에 맛있다는 얘기듣고 구매했어요포장 정말 맘에들어요ㅋ
12 평점5rico221026신고향이좋고 목넘김이 좋음
13 평점4ymjl221025신고좋아요좋아요좋아요좋아요
14 평점4gcjo221002신고괜찮은술이지만 다듬어져야함
15 평점5kaza220926신고배송 빠르고 좋아요
1 페이지 크롤링 완료------------------------------------------------------------------------


## 한 제품 df를 csv 저장 할 수 있는 코드 (테스트용)

In [10]:
import math

url ='https://smartstore.naver.com/woorisulgallery/products/6177890287'


header = {
     # referer --> 유저가 이전에 있던 위치를 알려주는 거, 안써주면 네이버가 정보 안준다
    'referer': 'https://finance.daum.net/quotes/A005930',
    # 나 기계아니고 사람임~ 내 노트북 정보
    'user_agent' : 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Mobile Safari/537.36',
}
#헤더 정보를 넣어주기
resp = requests.get(url, headers = header)
resp.text
html_text = BeautifulSoup(resp.text, 'html.parser')


# 리뷰 크롤링 위해서 크롬 창 열기
driver = set_chrome_driver()
driver.get(url)
time.sleep(2)


# 리뷰가 있을 경우만 크롤링
if int(html_text.select('strong._2pgHN-ntx6')[0].text) < 1 :    # 리뷰 1개 이하이면 pass
    pass

else :
    
    #제품명 
    product_info = [item.text.strip().replace('\t', '')  for item in html_text.select('div.CxNYUPvHfB')]
    print(product_info[0])
    
    
    count = 1
    stop = math.ceil(int(html_text.select('strong._2pgHN-ntx6')[0].text) / 20)  # 전체 리뷰 수 / 한 페이지 리뷰수 20개 

    review_list = []
    # 맨처음 리뷰 1페이지 누르고
    wait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#REVIEW > div > div._180GG7_7yx > div.cv6id6JEkg > div > div > a:nth-child(2)'))).click()
    while count < stop+1:
        # 페이지에 리뷰 수 구하기 위한 코드 : len() 
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        review = soup.find_all('div',class_='_1YShY6EQ56')
        for i in range(len(review)):
            review = soup.find_all('div',class_='_1YShY6EQ56')
            review = review[i].text
            review = re.sub('[^#0-9a-zA-Zㄱ-ㅣ가-힣 ]',"",review)
            review_list.append(review)        
            print(i, review)
        print(f'{count} 페이지 크롤링 완료------------------------------------------------------------------------')
        # 다음페이지 누르기
        if count == stop:
            break
        else:
            wait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#REVIEW > div > div._180GG7_7yx > div.cv6id6JEkg > div > div > a.fAUKm1ewwo._2Ar8-aEUTq'))).click()
            time.sleep(2)
            count = count + 1

# 데이터 프레임 만드는 함수 -> 실행하기 전에 함수 호출하기
woorigallery = woorigqllery_dataFrame(review_list, product_info)
woorigallery


영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르
0 평점5l221024신고포장도 꼼꽁하게 되어 배송되었고 우리나라 술들 골라먹는 재미가 있어서 좋아요
1 평점5oxou221014신고왜 도원결의일까요 술취하면 싸우기바쁜뎅ㅎㅎ 복숭아향 청하
2 평점5dbsk221019신고아직 먹기전인데 기대중이에여 ㅎㅎㅎ
3 평점5ohki221007신고포장 엄청 꼼꼼하게 와서 너무 좋네요배송도 정말 빨리 왔어요
4 평점42ano221021신고부드럽고 은은한 복숭아 향은 굿 달달한 맛은 조금 아쉽
5 평점5risa221014신고호랑이 디자인 갖구싶어서 샀으용ㅎㅎ
6 평점5als2221101신고한달사용기회식때 유용하게 잘 썼어요
7 평점5cia5220928신고호랑이 에디션 병이 예뻐서 샀어요
8 평점5diba220927신고항상잘먹고있습니다
9 평점5you8220905신고포장 꼼꼼히 해주셔서 감사합니다전통주 추석선물로도 넘 좋아용담에 또주문할게요
10 평점5dks4220927신고한달사용기매장에서 먹을땐 맛있어서 따로 주문했는데요 취해서 맛있엇나봐요시럽맛나요 ㅠ
11 평점5king220831신고첫 번째 주문하고 마음에 들어서추가로 주문했습니다우선 파손 없이 꼼꼼하게 포장되어안전한 병 배송이 마음에 듭니다다음에도 추가 구매 예정입니다더보기
12 평점5alee220930신고재구매살짝 느껴지는 복숭아 향과 맛이 좋아요
13 평점5tu220913신고목넘김도 좋고 맛도 좋아요
14 평점5diba221008신고재구매매번잘먹고있어요
15 평점522sa220914신고복숭아맛이라고 해서 주문해요
16 평점5tkdg220718신고지리산강쇠 가장 저렴하고 가장 기대값이 낮고 큰기대없이 3만원 채우기용으로 구매했지만 가장 맛있었고 엄청 부드러웠고 목넘김이 예술입니다 특히 목넘길때 알콜비린내가 1도 안느껴집니다도원결의 복숭아 리큐르라 그래서 최소한 볼스 피치리큐르 같은 느낌이려나 해서 구매해보았습니다정말 못먹을맛입니다 감기약에 복숭아 불량식품향 입힌느낌 거짓말안하고 한잔먹고 다버

,review,product_info
0,평점5l221024신고포장도 꼼꽁하게 되어 배송되었고 우리나라 술들 골라먹는 재미가...,영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르
1,평점5oxou221014신고왜 도원결의일까요 술취하면 싸우기바쁜뎅ㅎㅎ 복숭아향 청하,영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르
2,평점5dbsk221019신고아직 먹기전인데 기대중이에여 ㅎㅎㅎ,영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르
3,평점5ohki221007신고포장 엄청 꼼꼼하게 와서 너무 좋네요배송도 정말 빨리 왔어요,영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르
4,평점42ano221021신고부드럽고 은은한 복숭아 향은 굿 달달한 맛은 조금 아쉽,영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르
...,...,...
90,평점5lsk3220117신고아직 먹어보진 못했지만 디자인이 너무 이쁩니다,영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르
91,평점5nam3220121신고배송도 빠르고 만족합니다,영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르
92,평점40sou220204신고효소처리스테비아만 없음 좋았을텐요 ㅋㅋ,영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르
93,평점5thd7220124신고너무 맛있어서 재주문 할게용,영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르


# 리뷰 리스트 DF 만들기

In [8]:
# DF 만드는 함수
def woorigqllery_dataFrame(review_list, product_info):
    import pandas as pd
    woorigallery = pd.DataFrame(review_list, columns=['review'])
    woorigallery['product_info'] = product_info[0]
    return woorigallery 

In [9]:
woorigallery = woorigqllery_dataFrame(review_list, product_info)
woorigallery

,review,product_info
0,평점5l221024신고포장도 꼼꽁하게 되어 배송되었고 우리나라 술들 골라먹는 재미가...,영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르
1,평점5oxou221014신고왜 도원결의일까요 술취하면 싸우기바쁜뎅ㅎㅎ 복숭아향 청하,영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르
2,평점5dbsk221019신고아직 먹기전인데 기대중이에여 ㅎㅎㅎ,영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르
3,평점5ohki221007신고포장 엄청 꼼꼼하게 와서 너무 좋네요배송도 정말 빨리 왔어요,영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르
4,평점42ano221021신고부드럽고 은은한 복숭아 향은 굿 달달한 맛은 조금 아쉽,영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르
...,...,...
90,평점5lsk3220117신고아직 먹어보진 못했지만 디자인이 너무 이쁩니다,영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르
91,평점5nam3220121신고배송도 빠르고 만족합니다,영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르
92,평점40sou220204신고효소처리스테비아만 없음 좋았을텐요 ㅋㅋ,영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르
93,평점5thd7220124신고너무 맛있어서 재주문 할게용,영덕주조 2022년 호랑이 에디션 도원결의15 15도 360ml 복숭아 리큐르


## 리스트 CSV 저장 코드

In [ ]:
import csv 
fields = ['review_list', 'product_info']
rows = [review_list]

with open('test.csv', 'w', newline='') as file:
    write = csv.writer(file) 
    write.writerow(fields) 
    write.writerows(rows)